In [1]:
#전체 기사 내용에서 쓰이는 단어들의 tf-idf 값을 구해서 자주 쓰이는 쓸모없는 단어들을 수치로 표현해준다.
import sys
from konlpy.tag import Twitter
from konlpy.tag import Mecab
from collections import Counter
import os
import math
import re

In [111]:
#tfidf를 구하기 위해서 tf와 idf를 구하기 위한 시작 1글자 이상의 단어만
# 생각해볼점 한글자인 것들을 제거 한후 할 것인지 아니면 1글자들도 tf-idf 를 구해서 제거해서 할 것인지
# 제거 한 후 하는 것은 한 글자들은 대체로 의미가 없는 단어들이지만 필요한 단어가 있을 수도 있으며 좋은 점은 tf-idf 후 글자 clean의 시간을 단축시킴
# 제거 하지 않으면 실제로 필요한 단어들도 존재 할 수 있으며 자주쓰이는 단어는 당연히 골라 낼 수 있을 것이다. 하지만 clean 시간이 길어 진다.
def get_tags(text):
    spliter = Twitter()
    #spliter = Mecab()
    nouns = spliter.nouns(text)
    count = Counter(nouns)
    return_list = []
    for n, c in count.most_common():
        if len(n) > 1:
            temp = {'tag': n, 'count': c, 'dict_count' : 1}
            return_list.append(temp)
        #temp = {'tag': n, 'count': c, 'dict_count' : 1}
        #return_list.append(temp)
    return return_list

In [112]:
#tfidf가 0에 가까울 수록 좋지 제거 대상
#파라미터로는 각각의 단어에 대한 단어의 총 count와 어떤 문서에 쓰여져있는지의 count를 저장한 dict를 가져오고 총 문서의 수인 dict_num을 가져온다.
#많은 문서에서 쓰이는 단어 일수록 idf 값이 0에 가까워 지며 그로 인해 tf-idf 값이 0에 가까워 지면 제거 대상이다.
#로그빈도를 이용해서 tf값을 구했다. 값이 커지는 것을 막을 수 있다.
def tfidf_func(result_dict, dict_num):
    result = []
    for tag in result_dict:
        tf = math.log10(result_dict[tag][0] + 1)
        idf = math.log10(dict_num / result_dict[tag][1])
        tfidf = tf * idf
        #print(tag,result_dict[tag][0],result_dict[tag][1],tfidf)
        #tfidf 값을 조절해서 자주쓰이는 단어에 대해 알아낼수 있다.
        if tfidf < 1.9 and result_dict[tag][0] > 10:
            result.append(tag)
            print(tag,result_dict[tag][0],result_dict[tag][1],tfidf)
    #print(dict_num)
    #print(result)
    return result

In [113]:
#tfidf 하기
#처음에는 각각의 문서에 쓰여져 있는 단어의 count와 문서 count값을 가져와서 diction에 저장하고 계속해서 
def result_tfidf():
    num = 0
    #전체 문서에 쓰인 명사와 그 명사의 count, 명사의 문서에 쓰인 count 수를 저장하기 위한 변수
    result_dict = {}
    
    for (path, dir , files) in os.walk("article_dict"):
        #존재 여부 확인
        for fname in dir:
            fullname = path+"/"+fname
            #print(fullname)
            for file in os.listdir(fullname):
                #print(file)
                if file.endswith(".txt"):
                    open_text_file = open(fullname+"/"+file, 'r')
                    text = open_text_file.read()
            
                    #get_tags(text)를 통해 하나의 문서에 쓰인 명사와 그 명사의 count수 그리고 문서 count수인 1을 return 받는다.
                    tags = get_tags(text)
                    open_text_file.close()
            
                    #전체적인 명사와 count, dict_count를 저장해 result_dict에 그 값들을 저장한다. 
                    for tag in tags:
                        noun = tag['tag']
                        count = tag['count']
                        dict_count = tag['dict_count']
                        if noun in result_dict:
                            result_dict[noun][0] += count
                            result_dict[noun][1] += dict_count
                        else:
                            result_dict[noun] = [count, dict_count]
                    
                    #총 문서의 수를 저장하기 위해 사용하는 변수 하나의 문서를 처리할떄 마다 값을 1증가 시킨다.        
                    num = num + 1
                
    #위에서 계산한 result_dict와 전체 문서의 수인 num값을 파라미터로 넘겨준다.        
    return tfidf_func(result_dict, num)  

In [94]:
#tfidf를 통해 얻어온 제거 대상을 clean해주는 함수.
def clean_text(text,remove_list):
    cleaned_text = re.sub(remove_list, '', text)
    return cleaned_text

뉴스 5655 2496 1.3215682100063746
본문 10352 5224 0.12616916308625442
내용 11567 5224 0.12768354723407785
플레이어 10451 5224 0.12629904407013912
제보 3100 1717 1.7969251400353037
오류 5273 5224 0.11696427393399499
우회 5243 5224 0.11688642295408368
함수 5227 5224 0.11684472012630846
추가 6185 5224 0.11914100303186774
기자 7499 4360 0.4260265985156511
오후 1917 1545 1.8400320963282686
최신 1744 1741 1.6488689899520372
여러분 1513 1498 1.8251228503921457
클릭 2008 2003 1.4788965496387299
무단 3080 2995 0.9525184446985988
전재 2677 2677 1.102996525677194
배포 3081 3032 0.9339531772175607
금지 3706 3094 0.9240441803241378
서울 5432 2679 1.2006448932502884
페이스북 1852 1605 1.7775675739422254
국민 5404 2023 1.6552381368820093
오전 1855 1489 1.8844411071933034


In [109]:
#main function 각 문서들의 최종 clean된 명사들을 word2vec하기위해 하나의 txt파일에 저장한다.
def main():
    spliter = Twitter()
    remove_list = '|'.join(result_tfidf())
    #입력 파일명 = file
    for (path, dir , files) in os.walk("article_dict"):
        for fname in dir:
            fullname = path+"/"+fname
            print(fullname)
            for file in os.listdir(fullname):
                #존재 여부 확인
                if file.endswith(".txt"):
                    #출력 파일명
                    output_file_name = 'result.txt'
                    #output_file_name = 'konlpy/konlpy'+str(num)+'.txt'
                    print(fullname+"/"+file)
                    open_text_file = open(fullname+'/'+file, 'r')
                    text = open_text_file.read()
                    open_text_file.close()
            
                    #open_output_file = open(output_file_name, 'w')
                    open_output_file = open(output_file_name, 'a+')
            
                    result_text = clean_text(text,remove_list)
                    nouns = spliter.nouns(result_text)
                    
                    for n in nouns:
                        if len(n) > 1:
                            open_output_file.write(' ' + n)
                    open_output_file.write('\n')
                    open_output_file.close()

In [110]:
if __name__ == '__main__':
    main()

article_dict/20170806
article_dict/20170806/article_20170806_0.txt
article_dict/20170806/article_20170806_1.txt
article_dict/20170806/article_20170806_10.txt
article_dict/20170806/article_20170806_100.txt
article_dict/20170806/article_20170806_1000.txt
article_dict/20170806/article_20170806_1001.txt
article_dict/20170806/article_20170806_1002.txt
article_dict/20170806/article_20170806_1003.txt
article_dict/20170806/article_20170806_1004.txt
article_dict/20170806/article_20170806_1005.txt
article_dict/20170806/article_20170806_1006.txt
article_dict/20170806/article_20170806_1007.txt
article_dict/20170806/article_20170806_1008.txt
article_dict/20170806/article_20170806_1009.txt
article_dict/20170806/article_20170806_101.txt
article_dict/20170806/article_20170806_1010.txt
article_dict/20170806/article_20170806_1011.txt
article_dict/20170806/article_20170806_1012.txt
article_dict/20170806/article_20170806_1013.txt
article_dict/20170806/article_20170806_1014.txt
article_dict/20170806/articl

article_dict/20170806/article_20170806_1173.txt
article_dict/20170806/article_20170806_1174.txt
article_dict/20170806/article_20170806_1175.txt
article_dict/20170806/article_20170806_1176.txt
article_dict/20170806/article_20170806_1177.txt
article_dict/20170806/article_20170806_1178.txt
article_dict/20170806/article_20170806_1179.txt
article_dict/20170806/article_20170806_118.txt
article_dict/20170806/article_20170806_1180.txt
article_dict/20170806/article_20170806_1181.txt
article_dict/20170806/article_20170806_1182.txt
article_dict/20170806/article_20170806_1183.txt
article_dict/20170806/article_20170806_1184.txt
article_dict/20170806/article_20170806_1185.txt
article_dict/20170806/article_20170806_1186.txt
article_dict/20170806/article_20170806_1187.txt
article_dict/20170806/article_20170806_1188.txt
article_dict/20170806/article_20170806_1189.txt
article_dict/20170806/article_20170806_119.txt
article_dict/20170806/article_20170806_1190.txt
article_dict/20170806/article_20170806_119

article_dict/20170806/article_20170806_1341.txt
article_dict/20170806/article_20170806_1342.txt
article_dict/20170806/article_20170806_1343.txt
article_dict/20170806/article_20170806_1344.txt
article_dict/20170806/article_20170806_1345.txt
article_dict/20170806/article_20170806_1346.txt
article_dict/20170806/article_20170806_1347.txt
article_dict/20170806/article_20170806_1348.txt
article_dict/20170806/article_20170806_1349.txt
article_dict/20170806/article_20170806_135.txt
article_dict/20170806/article_20170806_1350.txt
article_dict/20170806/article_20170806_1351.txt
article_dict/20170806/article_20170806_1352.txt
article_dict/20170806/article_20170806_1353.txt
article_dict/20170806/article_20170806_1354.txt
article_dict/20170806/article_20170806_1355.txt
article_dict/20170806/article_20170806_1356.txt
article_dict/20170806/article_20170806_1357.txt
article_dict/20170806/article_20170806_1358.txt
article_dict/20170806/article_20170806_1359.txt
article_dict/20170806/article_20170806_13

article_dict/20170806/article_20170806_154.txt
article_dict/20170806/article_20170806_1540.txt
article_dict/20170806/article_20170806_1541.txt
article_dict/20170806/article_20170806_1542.txt
article_dict/20170806/article_20170806_1543.txt
article_dict/20170806/article_20170806_1544.txt
article_dict/20170806/article_20170806_1545.txt
article_dict/20170806/article_20170806_1546.txt
article_dict/20170806/article_20170806_1547.txt
article_dict/20170806/article_20170806_1548.txt
article_dict/20170806/article_20170806_1549.txt
article_dict/20170806/article_20170806_155.txt
article_dict/20170806/article_20170806_1550.txt
article_dict/20170806/article_20170806_1551.txt
article_dict/20170806/article_20170806_1552.txt
article_dict/20170806/article_20170806_1553.txt
article_dict/20170806/article_20170806_1554.txt
article_dict/20170806/article_20170806_1555.txt
article_dict/20170806/article_20170806_1556.txt
article_dict/20170806/article_20170806_1557.txt
article_dict/20170806/article_20170806_155

article_dict/20170806/article_20170806_1743.txt
article_dict/20170806/article_20170806_1744.txt
article_dict/20170806/article_20170806_1745.txt
article_dict/20170806/article_20170806_1746.txt
article_dict/20170806/article_20170806_175.txt
article_dict/20170806/article_20170806_176.txt
article_dict/20170806/article_20170806_177.txt
article_dict/20170806/article_20170806_178.txt
article_dict/20170806/article_20170806_179.txt
article_dict/20170806/article_20170806_18.txt
article_dict/20170806/article_20170806_180.txt
article_dict/20170806/article_20170806_181.txt
article_dict/20170806/article_20170806_182.txt
article_dict/20170806/article_20170806_183.txt
article_dict/20170806/article_20170806_184.txt
article_dict/20170806/article_20170806_185.txt
article_dict/20170806/article_20170806_186.txt
article_dict/20170806/article_20170806_187.txt
article_dict/20170806/article_20170806_188.txt
article_dict/20170806/article_20170806_189.txt
article_dict/20170806/article_20170806_19.txt
article_dic

...

article_dict/20170807/article_20170807_599.txt
article_dict/20170807/article_20170807_6.txt
article_dict/20170807/article_20170807_60.txt
article_dict/20170807/article_20170807_600.txt
article_dict/20170807/article_20170807_601.txt
article_dict/20170807/article_20170807_602.txt
article_dict/20170807/article_20170807_603.txt
article_dict/20170807/article_20170807_604.txt
article_dict/20170807/article_20170807_605.txt
article_dict/20170807/article_20170807_606.txt
article_dict/20170807/article_20170807_607.txt
article_dict/20170807/article_20170807_608.txt
article_dict/20170807/article_20170807_609.txt
article_dict/20170807/article_20170807_61.txt
article_dict/20170807/article_20170807_610.txt
article_dict/20170807/article_20170807_611.txt
article_dict/20170807/article_20170807_612.txt
article_dict/20170807/article_20170807_613.txt
article_dict/20170807/article_20170807_614.txt
article_dict/20170807/article_20170807_615.txt
article_dict/20170807/article_20170807_616.txt
article_dict/2017

article_dict/20170807/article_20170807_763.txt
article_dict/20170807/article_20170807_764.txt
article_dict/20170807/article_20170807_765.txt
article_dict/20170807/article_20170807_766.txt
article_dict/20170807/article_20170807_767.txt
article_dict/20170807/article_20170807_768.txt
article_dict/20170807/article_20170807_769.txt
article_dict/20170807/article_20170807_77.txt
article_dict/20170807/article_20170807_770.txt
article_dict/20170807/article_20170807_771.txt
article_dict/20170807/article_20170807_772.txt
article_dict/20170807/article_20170807_773.txt
article_dict/20170807/article_20170807_774.txt
article_dict/20170807/article_20170807_775.txt
article_dict/20170807/article_20170807_776.txt
article_dict/20170807/article_20170807_777.txt
article_dict/20170807/article_20170807_778.txt
article_dict/20170807/article_20170807_779.txt
article_dict/20170807/article_20170807_78.txt
article_dict/20170807/article_20170807_780.txt
article_dict/20170807/article_20170807_781.txt
article_dict/20

article_dict/20170807/article_20170807_927.txt
article_dict/20170807/article_20170807_928.txt
article_dict/20170807/article_20170807_929.txt
article_dict/20170807/article_20170807_93.txt
article_dict/20170807/article_20170807_930.txt
article_dict/20170807/article_20170807_931.txt
article_dict/20170807/article_20170807_932.txt
article_dict/20170807/article_20170807_933.txt
article_dict/20170807/article_20170807_934.txt
article_dict/20170807/article_20170807_935.txt
article_dict/20170807/article_20170807_936.txt
article_dict/20170807/article_20170807_937.txt
article_dict/20170807/article_20170807_938.txt
article_dict/20170807/article_20170807_939.txt
article_dict/20170807/article_20170807_94.txt
article_dict/20170807/article_20170807_940.txt
article_dict/20170807/article_20170807_941.txt
article_dict/20170807/article_20170807_942.txt
article_dict/20170807/article_20170807_943.txt
article_dict/20170807/article_20170807_944.txt
article_dict/20170807/article_20170807_945.txt
article_dict/20

article_dict/20170828/article_20170828_186.txt
article_dict/20170828/article_20170828_187.txt
article_dict/20170828/article_20170828_188.txt
article_dict/20170828/article_20170828_189.txt
article_dict/20170828/article_20170828_19.txt
article_dict/20170828/article_20170828_190.txt
article_dict/20170828/article_20170828_191.txt
article_dict/20170828/article_20170828_192.txt
article_dict/20170828/article_20170828_193.txt
article_dict/20170828/article_20170828_194.txt
article_dict/20170828/article_20170828_195.txt
article_dict/20170828/article_20170828_196.txt
article_dict/20170828/article_20170828_197.txt
article_dict/20170828/article_20170828_198.txt
article_dict/20170828/article_20170828_199.txt
article_dict/20170828/article_20170828_2.txt
article_dict/20170828/article_20170828_20.txt
article_dict/20170828/article_20170828_200.txt
article_dict/20170828/article_20170828_201.txt
article_dict/20170828/article_20170828_202.txt
article_dict/20170828/article_20170828_203.txt
article_dict/2017

article_dict/20170828/article_20170828_377.txt
article_dict/20170828/article_20170828_378.txt
article_dict/20170828/article_20170828_379.txt
article_dict/20170828/article_20170828_38.txt
article_dict/20170828/article_20170828_380.txt
article_dict/20170828/article_20170828_381.txt
article_dict/20170828/article_20170828_382.txt
article_dict/20170828/article_20170828_383.txt
article_dict/20170828/article_20170828_384.txt
article_dict/20170828/article_20170828_385.txt
article_dict/20170828/article_20170828_386.txt
article_dict/20170828/article_20170828_387.txt
article_dict/20170828/article_20170828_388.txt
article_dict/20170828/article_20170828_39.txt
article_dict/20170828/article_20170828_4.txt
article_dict/20170828/article_20170828_40.txt
article_dict/20170828/article_20170828_41.txt
article_dict/20170828/article_20170828_42.txt
article_dict/20170828/article_20170828_43.txt
article_dict/20170828/article_20170828_44.txt
article_dict/20170828/article_20170828_45.txt
article_dict/20170828/a